In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file = open('/content/drive/MyDrive/Other Files/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)

Error at line number:  281873
Error at line number:  281874
Error at line number:  281875


In [5]:
processedList[:100]

[['33', 'Jogging', '49105962326000', '-0.6946377', '12.680544', '0.50395286'],
 ['33', 'Jogging', '49106062271000', '5.012288', '11.264028', '0.95342433'],
 ['33', 'Jogging', '49106112167000', '4.903325', '10.882658', '-0.08172209'],
 ['33', 'Jogging', '49106222305000', '-0.61291564', '18.496431', '3.0237172'],
 ['33', 'Jogging', '49106332290000', '-1.1849703', '12.108489', '7.205164'],
 ['33', 'Jogging', '49106442306000', '1.3756552', '-2.4925237', '-6.510526'],
 ['33', 'Jogging', '49106542312000', '-0.61291564', '10.56939', '5.706926'],
 ['33', 'Jogging', '49106652389000', '-0.50395286', '13.947236', '7.0553403'],
 ['33', 'Jogging', '49106762313000', '-8.430995', '11.413852', '5.134871'],
 ['33', 'Jogging', '49106872299000', '0.95342433', '1.3756552', '1.6480621'],
 ['33', 'Jogging', '49106982315000', '-8.19945', '19.57244', '2.7240696'],
 ['33', 'Jogging', '49107092330000', '1.4165162', '5.7886477', '2.982856'],
 ['33', 'Jogging', '49107202316000', '-1.879608', '-2.982856', '-0.2996

In [6]:
columns = ['user', 'activity', 'time', 'x', 'y', 'z']
df = pd.DataFrame(data = processedList, columns = columns)
df.head()

,user,activity,time,x,y,z
0,33,Jogging,49105962326000,-0.6946377,12.680544,0.50395286
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,33,Jogging,49106222305000,-0.61291564,18.496431,3.0237172
4,33,Jogging,49106332290000,-1.1849703,12.108489,7.205164


In [7]:
df = df.drop(['user', 'time'], axis = 1).copy()
df.head()

,activity,x,y,z
0,Jogging,-0.6946377,12.680544,0.50395286
1,Jogging,5.012288,11.264028,0.95342433
2,Jogging,4.903325,10.882658,-0.08172209
3,Jogging,-0.61291564,18.496431,3.0237172
4,Jogging,-1.1849703,12.108489,7.205164


In [8]:
X = df[['x', 'y','z']].values
y = df['activity'].values

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [10]:
t = int(0.8*len(X))
X_train, X_test, y_train, y_test = X[:t], X[t:], y[:t], y[t:]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train.reshape(-1,1)).todense()
y_test = ohe.transform(y_test.reshape(-1,1)).todense()

**UNet1D**

In [11]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

class UNet1D(tf.keras.Model):
    """
    Adapted from https://keras.io/examples/vision/oxford_pets_image_segmentation/
    args
    num_classes : int
        Number of output classes, eg. P and S wave picking num_classes=2.
    """

    def __init__(self,
                 num_classes=2,
                 filters=None,
                 output_activation='linear',
                 kernel_regularizer=None,
                 dropout_rate=0.2,
                 initializer='glorot_normal',
                 name='UNet1D'):
        super(UNet1D, self).__init__(name=name)
        self.num_classes = num_classes
        self.initializer = initializer
        self.kernel_regularizer = kernel_regularizer
        self.dropout_rate = dropout_rate
        self.output_activation = output_activation

        if filters is None:
            self.filters = [4, 8, 16, 32]
        else:
            self.filters = filters

    def build(self, input_shape):
        inputs = tf.keras.Input(shape=input_shape[1:])

        ### [First half of the network: downsampling inputs] ###

        # Entry block
        x = tfl.Conv1D(self.filters[0], 7,
                       strides=2,
                       kernel_regularizer=self.kernel_regularizer,
                       padding="same",
                       name='entry')(inputs)

        x = tfl.BatchNormalization()(x)
        x = tfl.Activation("relu")(x)
        x = tfl.Dropout(self.dropout_rate)(x)

        previous_block_activation = x  # Set aside residual

        # Blocks 1, 2, 3 are identical apart from the feature depth.
        for i, filters in enumerate(self.filters[1:]):
            x = tfl.Activation("relu")(x)
            x = tfl.Conv1D(filters, 7, padding="same",
                           kernel_regularizer=self.kernel_regularizer,
                           kernel_initializer=self.initializer,
                           )(x)
            x = tfl.BatchNormalization()(x)
            x = tfl.Activation("relu")(x)
            x = tfl.Dropout(self.dropout_rate)(x)

            x = tfl.Conv1D(filters, 7, padding="same",
                           kernel_regularizer=self.kernel_regularizer,
                           kernel_initializer=self.initializer,
                           )(x)
            x = tfl.BatchNormalization()(x)

            x = tfl.MaxPooling1D(4, strides=2, padding="same")(x)

            # Project residual
            residual = tfl.Conv1D(filters, 1, strides=2, padding="same", kernel_initializer=self.initializer,
                                  )(
                previous_block_activation
            )
            x = tfl.concatenate([x, residual])  # Add back residual
            previous_block_activation = x  # Set aside next residual

        self.encoder = tf.keras.Model(inputs, x)
        ### [Second half of the network: upsampling inputs] ###

        for filters in self.filters[::-1]:
            x = tfl.Activation("relu")(x)
            x = tfl.Conv1DTranspose(filters, 7, padding="same",
                                    kernel_regularizer=self.kernel_regularizer,
                                    kernel_initializer=self.initializer,
                                    )(x)
            x = tfl.BatchNormalization()(x)
            x = tfl.Activation("relu")(x)
            x = tfl.Dropout(self.dropout_rate)(x)

            x = tfl.Conv1DTranspose(filters, 7, padding="same",
                                    kernel_regularizer=self.kernel_regularizer,
                                    kernel_initializer=self.initializer,
                                    )(x)
            x = tfl.BatchNormalization()(x)
            x = tfl.UpSampling1D(2)(x)

            # Project residual
            residual = tfl.UpSampling1D(2)(previous_block_activation)
            residual = tfl.Conv1D(filters, 1, padding="same",
                                  kernel_regularizer=self.kernel_regularizer,
                                  kernel_initializer=self.initializer,
                                  )(residual)
            x = tfl.concatenate([x, residual])  # Add back residual
            previous_block_activation = x  # Set aside next residual



        # Add a per-pixel classification layer
        if self.num_classes is not None:
            outputs = tfl.Conv1D(self.num_classes,
                           3,
                           padding="same",
                           activation=self.output_activation)(x)
        else:
            outputs = x

        # Define the model
        self.model = tf.keras.Model(inputs, outputs)

    def summary(self):
        return self.model.summary()

    def call(self, inputs):
        return self.model(inputs)

In [12]:
INPUT_SIZE = 128
model = UNet1D(num_classes=y_train.shape[1],
              output_activation='softmax')
model.build((None,INPUT_SIZE,3))
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
             metrics=['accuracy'])

In [13]:
class TimeSeriesInputOutputSeqeunce(tf.keras.utils.Sequence):

    def __init__(self, X, y,
                 batch_size=32,
                 input_size=(INPUT_SIZE,3),
                 length=INPUT_SIZE,
                 shuffle=True):

        self.batch_size = batch_size
        self.input_size = input_size
        self.length = length
        self.shuffle = shuffle
        self.X = X
        self.y = y

        self.on_epoch_end()

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.y)-self.length)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        idx = self.indexes[index:index+self.batch_size]
        X, y = [], []
        for i in idx:
            X.append(self.X[i:i+self.length])
            y.append(self.y[i:i+self.length])

        return np.asarray(X), np.asarray(y)

    def __len__(self):
        return len(self.y) // self.batch_size

In [14]:
train = TimeSeriesInputOutputSeqeunce(X_train, y_train)
model.fit(train, epochs=1, verbose=1)

8585/8585 [==============================] - 353s 40ms/step - loss: 0.2192 - accuracy: 0.9411


In [15]:
test = TimeSeriesInputOutputSeqeunce(X_test, y_test)



In [16]:
from sklearn.metrics import accuracy_score

ypred=model.predict(test)

2146/2146 [==============================] - 28s 13ms/step


In [17]:
ypred


array([[[9.46499407e-02, 2.30109971e-02, 2.53238846e-02, 2.41836775e-02,
         4.77158844e-01, 3.55672508e-01],
        [1.59988344e-01, 1.32787544e-02, 1.94515791e-02, 1.94145422e-02,
         5.72422147e-01, 2.15444714e-01],
        [1.65751085e-01, 1.34015456e-02, 1.95387360e-02, 2.05350854e-02,
         5.66365302e-01, 2.14408085e-01],
        ...,
        [1.17583193e-01, 2.23256405e-02, 7.35155772e-03, 2.05045175e-02,
         6.76210463e-01, 1.56024680e-01],
        [1.11857139e-01, 2.43555754e-02, 7.35466974e-03, 2.14359984e-02,
         6.69955015e-01, 1.65041551e-01],
        [2.92163968e-01, 3.50343399e-02, 2.56215222e-02, 2.61381809e-02,
         4.10224855e-01, 2.10817084e-01]],

       [[7.25591481e-01, 6.00314178e-02, 2.55740266e-02, 1.90895796e-02,
         4.18717638e-02, 1.27841651e-01],
        [9.30060089e-01, 6.97842473e-03, 2.07348168e-03, 2.68265815e-03,
         4.57093380e-02, 1.24959275e-02],
        [9.34414566e-01, 6.50590332e-03, 1.97918876e-03, 2.615024